<a href="https://colab.research.google.com/github/sipocz/powerplant/blob/B20210606_Submission13/powerplant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
weblink_train="https://raw.githubusercontent.com/sipocz/powerplant/data/TRAIN.csv"
fname_train=weblink_train.split("/")[-1]
!rm $fname_train
!wget $weblink_train

--2021-06-06 10:55:29--  https://raw.githubusercontent.com/sipocz/powerplant/data/TRAIN.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 258934 (253K) [text/plain]
Saving to: ‘TRAIN.csv’

TRAIN.csv           100%[===================>] 252.87K  --.-KB/s    in 0.02s   

2021-06-06 10:55:30 (10.9 MB/s) - ‘TRAIN.csv’ saved [258934/258934]



In [3]:
df=pd.read_csv(fname_train)

In [4]:
df.head()

,AT,V,AP,RH,PE
0,22.85,60.29,1017.29,61.92,448.58
1,11.75,35.76,1018.99,57.92,473.01
2,12.24,44.90,1020.31,82.22,464.25
3,13.43,40.10,1015.82,87.90,471.77
4,4.65,35.19,1018.23,94.78,489.36


In [5]:
corr=df.corr()
print(corr)

          AT         V        AP        RH        PE
AT  1.000000  0.843584 -0.505347 -0.538296 -0.947350
V   0.843584  1.000000 -0.409484 -0.308251 -0.869313
AP -0.505347 -0.409484  1.000000  0.100138  0.515031
RH -0.538296 -0.308251  0.100138  1.000000  0.385353
PE -0.947350 -0.869313  0.515031  0.385353  1.000000


In [6]:
import seaborn as sns
#sns.pairplot(df)

In [7]:
X=df.iloc[:,:-1]

In [8]:
y=df.iloc[:,-1]

In [9]:
y

0       448.58
1       473.01
2       464.25
3       471.77
4       489.36
         ...  
7995    456.25
7996    468.35
7997    465.14
7998    439.73
7999    481.46
Name: PE, Length: 8000, dtype: float64

In [10]:

from sklearn import preprocessing

x = X.values #returns a numpy array
min_max_scaler_X = preprocessing.MinMaxScaler()
X_scale = min_max_scaler_X.fit_transform(x)
X_scaled = pd.DataFrame(X_scale)

y_values=y.values
y_values=y_values.reshape(-1,1)
min_max_scaler_y = preprocessing.MinMaxScaler()
y_scale = min_max_scaler_y.fit_transform(y_values)
y_scaled = pd.DataFrame(y_scale)



In [11]:
y_values.reshape(-1,1)

array([[448.58],
       [473.01],
       [464.25],
       ...,
       [465.14],
       [439.73],
       [481.46]])

In [12]:
X_scaled.head()

,0,1,2,3
0,0.589876,0.621530,0.601642,0.487399
1,0.270636,0.185053,0.643941,0.433780
2,0.284728,0.347687,0.676785,0.759517
3,0.318953,0.262278,0.565066,0.835657
4,0.066437,0.174911,0.625031,0.927882


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.1, random_state=0)

In [14]:
y_test

,0
3069,0.094570
1675,0.146490
6385,0.298411
543,0.827550
3213,0.345430
...,...
3890,0.091921
6650,0.288344
938,0.377086
527,0.725828


In [15]:
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model
from keras.optimizers import SGD,Adam,Adamax,Nadam,Ftrl,Adadelta,Adagrad,Nadam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences

In [39]:
clear_session()



input_len=len(X_train.columns)

print(input_len)
output_size=1
drop_frac0=0.0 
drop_frac1=0.0



input1=Input(shape=(input_len,),)

#flatt=Flatten()(lstm1)

non=42
#initializer = tf.keras.initializers.LecunNormal()
#initializer=tf.keras.initializers.LecunUniform()
#initializer=tf.keras.initializers.HeUniform(    seed=None)
#initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

initializer="normal"
d1=Dense(193,activation="sigmoid",kernel_initializer=initializer,input_dim=input_len)(input1)
drop1=Dropout(drop_frac0)(d1)

d2=Dense(120,activation="sigmoid",kernel_initializer=initializer)(drop1)
drop2=Dropout(drop_frac1)(d2)
d3=Dense(193,activation="sigmoid",kernel_initializer=initializer)(drop2)
drop3=Dropout(drop_frac0)(d3)


pred=Dense(output_size,activation="sigmoid")(drop3)

model = Model(inputs=input1, outputs=pred)

opt = tf.keras.optimizers.Adam(learning_rate=0.001)


rmse = tf.keras.metrics.RootMeanSquaredError()
model.compile(loss="mean_squared_error",
    optimizer=opt,
    metrics=[rmse])

4


In [40]:
!rm a1*

In [33]:
def scheduler(epoch, lr):
    return 0.001

In [19]:
callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [20]:
model_name="a1"

In [41]:
lista=['1',"2",'1',"2",'1',"2",'1',"2",'1',"2"]
def savemodela(sheduler):
    modelnamestr=model_name+"_{val_root_mean_squared_error:.5f}__.hdf5"
    lista.append(modelnamestr)
    remover=lista[0]
    lista=lista[1:]
    !rm $remover

callbacks = [callback_LR,
        
        #savemodela,
        ModelCheckpoint(filepath=model_name+"_{val_root_mean_squared_error:.5f}__.hdf5", monitor='val_root_mean_squared_error',
                        verbose=2, save_best_only=True, mode='min')]

In [22]:
#modelname="_model_1__1.68411__.hdf5"
#model=tf.keras.models.load_model(modelname)

In [23]:
'''
modelname="_model_1__1.80573__.hdf5"
model=tf.keras.models.load_model(modelname)
'''

'\nmodelname="_model_1__1.80573__.hdf5"\nmodel=tf.keras.models.load_model(modelname)\n'

In [43]:
history = model.fit(    X_train,
                        y_train,
                        validation_data=(X_test,y_test), 
                        epochs=555, 
                        batch_size=2,validation_split=0.0,
                        verbose=1,
                        callbacks=callbacks,shuffle=False,
                              
                    )

Epoch 1/555
3600/3600 [==============================] - 7s 2ms/step - loss: 0.0026 - root_mean_squared_error: 0.0546 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0546

Epoch 00001: val_root_mean_squared_error improved from 0.05458 to 0.05457, saving model to a1_0.05457__.hdf5
Epoch 2/555
3600/3600 [==============================] - 7s 2ms/step - loss: 0.0028 - root_mean_squared_error: 0.0546 - val_loss: 0.0034 - val_root_mean_squared_error: 0.0546

Epoch 00002: val_root_mean_squared_error improved from 0.05457 to 0.05457, saving model to a1_0.05457__.hdf5
Epoch 3/555
3600/3600 [==============================] - 7s 2ms/step - loss: 0.0027 - root_mean_squared_error: 0.0546 - val_loss: 0.0033 - val_root_mean_squared_error: 0.0546

Epoch 00003: val_root_mean_squared_error improved from 0.05457 to 0.05457, saving model to a1_0.05457__.hdf5
Epoch 4/555
3600/3600 [==============================] - 7s 2ms/step - loss: 0.0026 - root_mean_squared_error: 0.0546 - val_loss: 0.0038 - val_ro

In [44]:
datag=pd.DataFrame()
y_pred=model.predict(X_train)


In [45]:
y_pred[1:3]

array([[0.63490355],
       [0.21274814]], dtype=float32)

In [46]:
weblink_test="https://raw.githubusercontent.com/sipocz/powerplant/data/TEST.csv"
fname_test=weblink_test.split("/")[-1]
!rm $fname_test
!wget $weblink_test

--2021-06-06 15:08:11--  https://raw.githubusercontent.com/sipocz/powerplant/data/TEST.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39937 (39K) [text/plain]
Saving to: ‘TEST.csv’

TEST.csv            100%[===================>]  39.00K  --.-KB/s    in 0.002s  

2021-06-06 15:08:12 (17.5 MB/s) - ‘TEST.csv’ saved [39937/39937]



In [47]:
df_pred=pd.read_csv(fname_test)

In [48]:
df_pred.head()

,AT,V,AP,RH
0,11.55,40.60,1014.44,87.06
1,6.61,38.91,1015.77,92.31
2,16.97,42.86,1013.92,74.80
3,21.90,57.76,1017.45,66.89
4,9.73,40.22,1011.48,87.88


In [49]:
X_pred=df_pred

In [50]:

x_pred = X_pred.values #returns a numpy array
X_pred_scale = min_max_scaler_X.fit_transform(x_pred)
X_pred_scaled = pd.DataFrame(X_pred_scale)




In [51]:
X_pred_scaled.describe()

,0,1,2,3
count,1568.000000,1568.000000,1568.000000,1568.000000
mean,0.524688,0.528311,0.511718,0.611043
std,0.221458,0.233787,0.150597,0.214616
min,0.000000,0.000000,0.000000,0.000000
25%,0.346593,0.297154,0.407842,0.460700
50%,0.552741,0.487413,0.499625,0.636344
75%,0.702889,0.750274,0.616446,0.783665
max,1.000000,1.000000,1.000000,1.000000


In [52]:
y_pred_scale=model.predict(X_pred_scale)

In [53]:
y_pred=min_max_scaler_y.inverse_transform(y_pred_scale)

In [54]:
y_pred

array([[471.15646],
       [488.367  ],
       [464.14554],
       ...,
       [477.58783],
       [459.531  ],
       [480.4034 ]], dtype=float32)

In [55]:
data=pd.DataFrame(X_pred.index)

In [56]:
data["Value"]=y_pred

In [57]:
data.columns=["Index","Value"]

In [58]:
data.head()

,Index,Value
0,0,471.156464
1,1,488.367004
2,2,464.145538
3,3,450.854950
4,4,479.290558


In [59]:
subfile="Submission13.csv"

In [60]:
data.to_csv(subfile,index=False)

In [61]:
!head $subfile

Index,Value
0,471.15646
1,488.367
2,464.14554
3,450.85495
4,479.29056
5,437.04675
6,449.2525
7,468.31827
8,466.6353
